# pre_processing

In [12]:
import pandas as pd
import numpy as np

In [13]:
import nbimporter
import feature_generation

In [14]:
features = feature_generation.get_features()
features_types = feature_generation.get_features_types()

In [15]:
def load_appended_dataset():
    """Carga ambos dataframes (train y test) y devuelve un solo dataframe que es el resultado de realizar un
    append entre ambos. El dataset de test esta al final."""

    train = pd.read_csv('./data/train.csv', index_col='id', parse_dates=['fecha'])
    test = pd.read_csv('./data/test.csv', index_col='id', parse_dates=['fecha'])
    df = train.append(test, sort=False)
   
    return df

In [16]:
def separate_train_test(df):
    """Separa df compuesto por sus primeras 240.000 filas de train.csv y sus 60.000 ultimas filas de test
    en los dos dataframes correspondientes."""
    
    train = df.head(240000)
    test = df.tail(60000).drop('precio', axis=1)
    return train, test

In [17]:
def load_datasets():
    """Carga ambos dataframes (train y test) y devuelve dos datasets en el formato:
    train_df, test_df"""

    df = load_appended_dataset()
    return separate_train_test(df)

In [18]:
def load_featured_appended_dataset():
    """Carga el .csv con los features generados, y lo formatea para ahorrar memoria. Devuelve un unico
    dataframe cuyas primeras 240.000 filas son de train.csv y sus ultimas 60.000 de test."""
      
    df = load_appended_dataset()
    
    csv_names = feature_generation.get_csvs_names()
    features_types = feature_generation.get_features_types()
    
    for file in csv_names:
        ruta = 'features/'+file+'.csv'
        add = pd.read_csv(ruta, index_col='id')
        df = pd.merge(df, add, how='left', on='id')
           
    for categoria in features_types.keys():
        for tipo in features_types[categoria]:
            castear = features_types[categoria][tipo]
            if (tipo == 'bernoulli'):
                df[castear] = df[castear].astype('uint8')
            elif (tipo == 'float'):
                df[castear] = df[castear].astype('float')
            elif (tipo == 'uint8'):
                df[castear] = df[castear].astype('uint8')
            elif (tipo == 'uint16'):
                df[castear] = df[castear].astype('uint16')
            elif (tipo == 'uint32'):
                df[castear] = df[castear].astype('uint32')
    
    return df

In [9]:
def load_featured_datasets():
    """Devuelve dos datasets en el formato:
    train_df, test_df
    Para los datasets con las features generadas."""
    
    df = load_featured_appended_dataset()
    return separate_train_test(df)

# Pre procesamiento de datasets externos
### Cotizacion USD

In [9]:
usd = pd.read_csv('external_datasets/usd_mxn_diario.csv')

#### funciones aux

In [10]:
def aniomes(x):
    anio=x[6:]
    mes=x[3:5]
    return anio+mes

def numeric(x):
    entero, fraccion = x.split(',')
    return int(entero) + int(fraccion)/10000

def varianza(x):
    x = x.rstrip('%')
    entero, fraccion = x.split(',')
    num = entero+"."+fraccion
    return float(num)

In [11]:
usd['aniomes'] = usd['Fecha'].map(aniomes)

In [12]:
valores = ['Último', 'Apertura', 'Máximo', 'Mínimo']
for valor in valores:
    usd[valor] = usd[valor].map(numeric)
    
usd['usd_varianza'] = usd['% var.'].map(varianza)
usd['daily_mean'] = usd.apply(lambda x: (x['Último'] + x['Apertura'])/2, axis=1)

In [13]:
usd = usd[['aniomes', 'usd_varianza', 'daily_mean']]

In [14]:
usd = usd.groupby('aniomes')['usd_varianza', 'daily_mean'].agg({'usd_varianza':'sum', 'daily_mean':'mean'})

In [15]:
usd.head()

,usd_varianza,daily_mean
aniomes,,
201201,-6.71,13.419316
201202,-1.50,12.789329
201203,-0.27,12.734559
201204,1.67,13.047955
201205,9.99,13.630126


In [16]:
usd.to_csv('data/usd_mxn_featured.csv')

# Volcanes

In [25]:
df = pd.read_csv('external_datasets/volcanes.csv')

In [26]:
df = df[['properties/OBJECTID', 'properties/LAT', 'properties/LONG_']]

In [27]:
df.columns = ['id', 'lat', 'long']

In [28]:
df.set_index('id', inplace=True)

In [29]:
df

,lat,long
id,,
19,19.030365,-97.268213
20,18.562021,-95.198504
21,19.493241,-102.251431
22,18.973674,-101.717852
23,19.308206,-110.806474
...,...,...
62,20.830000,-104.820000
59,28.500000,-113.750000
54,29.338829,-114.517599


In [30]:
df.to_csv('data/volcanes_featured.csv')

# Llenado de valores nulos en tipo de propiedad
Funcion que es llamada desde feature_generation

In [2]:
def buscar_tipo(x):
    lista = ['casas', 'departamento', 'casa', 'terrenos', 'terreno', 'departamentos',
             'apartamento', 'apartamentos', 'comercio', 'locales', 'condominio', 'residencia']
    palabras = x.lower().split(' ')
    categorias=[]
    for palabra in palabras:
        if palabra in lista:
            categorias.append(palabra)
    if len(categorias)==0:
        return ['Otros']
    elif len(categorias)>1:
        return casas_condominio(categorias)
    return categorias

def casas_condominio(lista):
    for palabra in lista:
        if palabra == 'condominio':
            return ['condominio']
    return lista

def translate_tiposdepropiedad(x):
    translate = {'casas':'Casa', 'departamento':'Apartamento', 'casa':'Casa', 'terrenos':'Terreno',
                 'terreno':'Terreno', 'departamentos':'Apartamento', 'apartamento':'Apartamento',
                 'apartamentos':'Apartamento', 'comercio':'Comercial', 'locales':'Comercial',
                 'condominio':'Casa en condominio', 'residencia':'Casa', 'Otros':'Otros'}
    return translate[x]

In [1]:
def fill_tipodepropiedad_nulls(nulos):
    nulos['tipodepropiedad_titulo'] = nulos['titulo'].map(buscar_tipo)
    nulos['tipodepropiedad_titulo'] = nulos['tipodepropiedad_titulo'].map(lambda x: translate_tiposdepropiedad(x[0]))
    
    return nulos.drop('tipodepropiedad', axis=1)